In [ ]:
''' This test script builds and runs three predictive models. The model with the lowest MSE score is
    used to show the most important features'''

__author__ = 'skyler bullard'

__copyright__ = 'Copyright 2018, Data Science Dream Job LLC'